In [3]:
import pandas as pd
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data as Data

from torch.autograd import Variable
from warpctc_pytorch import CTCLoss
import sys

In [4]:
sys.path.insert(1,'./crnn.pytorch')

In [5]:
import models.crnn as crnn

In [6]:
class strLabelConverter(object):
    """Convert between str and label.
    NOTE:
        Insert `blank` to the alphabet for CTC.
    Args:
        alphabet (str): set of the possible characters.
    """

    def __init__(self, alphabet, ignore_case=True):
        self._ignore_case = ignore_case
        if self._ignore_case:
            alphabet = alphabet.lower()
        self.alphabet = alphabet + '-'  # for `-1` index

        self.dict = {}
        for i, char in enumerate(alphabet):
            # NOTE: 0 is reserved for 'blank' required by wrap_ctc
            self.dict[char] = i + 1

In [7]:
alphabet = '零壹貳參肆伍陸柒捌玖拾佰仟萬億兆元整'
alphabet = alphabet + '-'
alphabet_dict = {}
for i, char in enumerate(alphabet):
    # NOTE: 0 is reserved for 'blank' required by wrap_ctc
    alphabet_dict[char] = i + 1

In [8]:
df = pd.read_pickle('../datamartV1.pkl')

In [7]:
df.head()

,root,Amount,image,Amount_hw,image_Small
0,/image/data/GF10244160490032.jpg,16000,"[[0.7254901960784313, 0.7176470588235294, 0.70...",壹萬陸仟元整,"[[0.7205575980392156, 0.6301623774509804, 0.62..."
1,/image/data/GF11241974690049.jpg,2468,"[[0.5254901960784314, 0.49411764705882355, 0.5...",貳仟肆佰陸拾捌元整,"[[0.6123468137254902, 0.8616268382352941, 0.73..."
2,/image/data/GF11012970590053.jpg,9545,"[[0.7294117647058823, 0.7058823529411765, 0.66...",玖仟伍佰肆拾伍元整,"[[0.6418504901960784, 0.8431066176470589, 0.70..."
3,/image/data/GF10336156190068.jpg,16870,"[[0.8470588235294118, 0.8470588235294118, 0.82...",壹萬陸仟捌佰柒拾元整,"[[0.5708026960784315, 0.7798253676470588, 0.66..."
4,/image/data/GF11366206090092.jpg,9000,"[[0.9019607843137255, 0.9647058823529412, 0.93...",玖仟元整,"[[0.8944087009803922, 0.8486366421568629, 0.85..."


In [9]:
df['Label'] = df.Amount_hw.apply(lambda x : np.array([alphabet_dict[i] for i in x]))

In [10]:
X = np.array([i.reshape(1,32,320) for i in df.image_Small.values])
y = np.array([i for i in df.Label.values])

In [13]:
X = torch.FloatTensor(X).type(torch.cuda.FloatTensor)

In [12]:
y = torch.FloatTensor(y).type(torch.cuda.FloatTensor)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: double, float, float16, int64, int32, and uint8.

In [ ]:
X = Variable(X)

In [13]:
X

array([[[[0.60189951, 0.43137255, 0.45009191, ..., 0.65294118,
          0.71352635, 0.77058824],
         [0.75073529, 0.68895527, 0.66236213, ..., 0.65294118,
          0.71053922, 0.77083333],
         [0.70863971, 0.69319853, 0.70292586, ..., 0.667356  ,
          0.69921875, 0.76530331],
         ...,
         [0.6379902 , 0.68871017, 0.62867647, ..., 0.61495098,
          0.63086703, 0.70914522],
         [0.6560815 , 0.64401042, 0.61668199, ..., 0.61617647,
          0.62608762, 0.71098346],
         [0.64436275, 0.66985294, 0.62311581, ..., 0.60188419,
          0.61847426, 0.65886949]]],


       [[[0.54316789, 0.88434436, 0.40617341, ..., 0.79681373,
          0.81793811, 0.79727328],
         [0.83540135, 0.80251225, 0.37161458, ..., 0.85004596,
          0.81541054, 0.80073529],
         [0.79898897, 0.76311275, 0.80386029, ..., 0.81574755,
          0.82768076, 0.78651961],
         ...,
         [0.81636029, 0.84276961, 0.78465074, ..., 0.77745098,
          0.77032782, 0

In [11]:
y

array([array([ 5, 14, 17, 18]), array([ 4, 14, 17, 18]),
       array([ 5, 14, 17, 18]), ...,
       array([ 2, 12,  6, 11,  5, 14,  9, 13,  5, 12,  4, 11,  6, 17, 18]),
       array([ 2, 12,  2, 11, 10, 14,  9, 13,  6, 12,  6, 11,  8, 17, 18]),
       array([ 4, 13,  5, 12, 10, 11, 14,  2, 13,  8, 12,  4, 11,  3, 17, 18])],
      dtype=object)

In [ ]:
print(X.size(),y.size())

In [ ]:
torch_dataset = Data.TensorDataset(X,y)

In [ ]:
X.shape

In [ ]:
#variable = Variable(X, requires_grad=True)

In [ ]:
batchSize = 64

In [ ]:
train_loader = Data.DataLoader(torch_dataset, batch_size=batchSize,
                                           shuffle=True,num_workers=4)

In [ ]:
criterion = CTCLoss()

In [ ]:
model = crnn.CRNN(32, 1, 19, 256)
model = model.cuda()
model.load_state_dict(torch.load("./crnn.pytorch/data/crnnV2.pkl"))

In [ ]:
model

In [ ]:
image = torch.FloatTensor(batchSize, 1, 32, 32)
text = torch.IntTensor(batchSize * 5)
length = torch.IntTensor(batchSize)

In [ ]:
image = image.cuda()
criterion = criterion.cuda()

In [ ]:
model = torch.nn.DataParallel(model)
image = image.cuda()
criterion = criterion.cuda()

In [ ]:
image = Variable(image)
text = Variable(text)
length = Variable(length)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.01126,betas=(0.01, 0.999))

In [ ]:
for epoch in range(1000):
    train_iter = iter(train_loader)
    i = 0
    while i < len(train_loader):
        for p in model.parameters():
            p.requires_grad = True
        model.train()

        i += 1

        if i % 10 == 0:
            print('[iteration%d/%d][%d/%d] ' %
                  (epoch, 1000, i, len(train_loader), ))

In [ ]:
torch.save(model.state_dict(), './crnn.pytorch/data/crnnV2.pkl')

In [ ]:
model

In [ ]:
from PIL import Image
Image.open('./crnn.pytorch/data/GF10222057890039.jpg')
